In [20]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json 
import requests
import datetime
from datetime import date
import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 800)
import random

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [3]:
x = newsapi.get_top_headlines(country='us', page_size=50, category = 'health')

In [4]:
nyt_api_key = '13bd501bc77542a58e2e6678619b0d60'

In [5]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

# NYT & NewsAPI

In [6]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
import datetime
from datetime import date
from apikeys import *
from info import *

#dates to use for API call
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

#NEW YORK TIMES
#clean the response from NYT API
def NYT_title_clean(df):
    titles = []
    for index, row in df.iterrows():
        title = row.headline['main']
        titles.append(title)
    df['title'] = titles
    return df

def NYT_dropped_rows(df):
    df.pub_date = pd.to_datetime(df.pub_date).dt.date
    df.word_count = round(df.word_count / 150)
    df.document_type = 'text'
    df['formality'] = 'Intermediate'
    return df

def NYT_dataframe_clean(df):
    dataframe = NYT_title_clean(df)
    dataframe = NYT_dropped_rows(dataframe)
    return dataframe

def NYT_api_call_section_based(section, source, page, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:({section_name})&page={page}&source:({source})&begin_date={start}&end_date={end}&api-key={api}'.format(section_name = section, page = page, source = source, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = section
    df['image_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHEtiVXw8Wi1tp56Nzd5rH_EoOAJA2RInEWvf5h5CQ-6O_YZp7dw'
    return df

def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = 'https://greaterbostonhcs.com/wp-content/uploads/2016/05/Nutrition.jpg'
    return df

def NYT_pull(categories):
    empty = pd.DataFrame()
    for word in categories:
        try:
            df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
            empty = empty.append(df, sort=True)
            print('Pulled '+word)
            time.sleep(2)
        except:
            print(word + " EXCEPTION!!!!")
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

#NEWSAPI

def rename_columns(df):
    df = df.rename(index=str, columns={'publishedAt':'date','url':'web_url','urlToImage':'image_url'})
    return df

def add_words(df):
    lengths = []
    for string in df.content:
        try:
            lengths.append(round(int(string[string.find('+')+1:string.find(' chars')]) / 4 / 250))
        except:
            lengths.append(4)
    return lengths

def split_source_info(list_of_dicts):
    for item in list_of_dicts:
        item['source_id'] = item['source']['id']
        item['source'] = item['source']['name']

def pull_articles(parameter):
    article_results_rel = newsapi.get_everything(q=parameter,sort_by = 'relevancy',language='en', page_size=10, sources=sources_joined)
    article_results_rel = article_results_rel['articles']
    split_source_info(article_results_rel)
    return article_results_rel

def clean_articles(list_of_dicts, search_param):
    df = pd.DataFrame(list_of_dicts)
    try:
        df['medium'] = 'text'
        df['param'] = search_param
        df['publishedAt'] = df['publishedAt'].apply(lambda x: pd.to_datetime(x).date().strftime('%Y-%m-%d'))
        df['formality'] = 'Intermediate'
        df['length'] = add_words(df)
        df = rename_columns(df)
        print(search_param)
    except:
        pass
    return df


def call_news_api(categories):
    empty_df = pd.DataFrame()
    for category in categories:
        dicts = pull_articles(category)
        df = clean_articles(dicts, category)
        try:
            empty_df = empty_df.append(df, sort=True)
        except:
            pass
        print(len(empty_df.index))
    return empty_df


# Social Media

In [140]:
nutrition_cats = ['aminos','keto','pressure cooked carbs','sprouted carbs','mct','coconut','farm','corn-fed','amino','cruciferous','wheat','vegetiable','cooking','factory','anti-biotic','grass-fed','plant','animal','blood','refined','simple','complex','white','brown','foodmap'
,'gluten','vegan','vegetarian','gut','paleo','ketogenic','infloamation','vegetiable oil','organic','gluten free','vegan diet','vegetarian diet','intermittent fasting','night shade','root veggies','cruciferous vegetables','grains','glysophate','pesticides','factory farming','free range','anti-biotic free','sprouts','beans','legumes'
,'lectins','dark leafy greens','foodmap diet','paleo diet','ketogenic diet','micronutrient','macronutrient','diabetes','meal prep','leaky gut','fat on nutrion label','lean protein','cholestrol','natural sugars','brain fog','mct oil','coconut oil','pressure cooked carbs','sprouted carbs','sourdough','bcaas','essential amino acids','wild fish','farm raised fish','corn-fed beef'
,'wild game','bioavailability','amino acids','meat','sprouted','vitamins','supplements','auto-immune','hormone','gluconeogenesis','bad fat','good fat','anti-inflammatory','inflammation','energy','mood','gut brain connection','wheat belly','fermented','vegetable oils','cooking oils','fat loss','grass-fed dairy','grass-fed protein','plant based protein'
,'animal protein','protein','sugar','blood glucose','fiber','starch','refined sugar','refined carbs','simple carbs','complex carbs','carbs','white hdl fat','brown ldl fat','fats','prebiotic','enzymes','probiotic','microbiome','gut health']


In [159]:
import tweepy
from apikeys import *
import pandas as pd
import json
import requests
import datetime
from datetime import date
from info import *

auth = tweepy.OAuthHandler(twitter_1, twitter_2)
auth.set_access_token(twitter_3, twitter_4)
api = tweepy.API(auth)
#all twitter handles to scrape
twitter_handles = ['@ATPScience1', '@waitrose', '@MicrobiomeInst']#, '@veganrecipescom', '@cldiet', '@Onnit', '@vegsoc', '@VeganKosher', '@TheVeganSociety', '@vegan', '@Keto_Recipes_', '@the52diet', '@IFdiet', '@microbiome', '@metagenomics', '@microbiome_news', '@TheGutStuff', '@MyGutHealth', '@PaleoFX',
# '@PaleoFoundation', '@ThePaleoDiet', '@PaleoComfort', '@cavemanketo', '@KetoFlu', '@TheKetoKitchen_', '@EatKetoWithMe', '@KetoConnect', '@KetoDietZone', '@Ketogenic', '@USDANutrition', '@FoodRev', '@CSPI', '@simplyrecipes', '@FoodNetwork', '@CookingChannel', '@tasty', '@nytfood', '@finecooking', '@mrcookingpanda'
# , '@FODMAPeveryday', '@FODMAPLife', '@FodmappedInfo', '@thefodmapdoctor', '@SimplyGlutenFre', '@gfliving', '@sibotest', '@manjulaskitchen', '@VegTimes', '@CookingLight', '@mealprepwl', '@thehealthygut', '@VitalGutHealth', '@pureguthealth', '@PaleoForBegin', '@PaleoLeap', '@ThePaleoMom', '@paleomagazine', '@PaleoHacks', '@paleogrubs',
# '@naturalgourmet', '@Low_Carb_Keto', '@NutritionTwins', '@mckelhill', '@WomensFitnessAu', '@WomensHealthMag', '@MensHealthMag', '@mjfit', '@thugkitchen', '@Leslie_Klenke', '@insidePN', '@ThisMamaCooks', '@EdibleWildFood', '@TheEarthDieter', '@HarvardHealth', '@EverydayHealth', '@DailyHealthTips']

#clean response from twitter
nutrition_cats = ['aminos','keto','pressure cooked carbs','sprouted carbs','mct','coconut','farm','corn-fed','amino','cruciferous','wheat','vegetiable','cooking','factory','anti-biotic','grass-fed','plant','animal','blood','refined','simple','complex','white','brown','foodmap'
,'gluten','vegan','vegetarian','gut','paleo','ketogenic','infloamation','vegetiable oil','organic','gluten free','vegan diet','vegetarian diet','intermittent fasting','night shade','root veggies','cruciferous vegetables','grains','glysophate','pesticides','factory farming','free range','anti-biotic free','sprouts','beans','legumes'
,'lectins','dark leafy greens','foodmap diet','paleo diet','ketogenic diet','micronutrient','macronutrient','diabetes','meal prep','leaky gut','fat on nutrion label','lean protein','cholestrol','natural sugars','brain fog','mct oil','coconut oil','pressure cooked carbs','sprouted carbs','sourdough','bcaas','essential amino acids','wild fish','farm raised fish','corn-fed beef'
,'wild game','bioavailability','amino acids','meat','sprouted','vitamins','supplements','auto-immune','hormone','gluconeogenesis','bad fat','good fat','anti-inflammatory','inflammation','energy','mood','gut brain connection','wheat belly','fermented','vegetable oils','cooking oils','fat loss','grass-fed dairy','grass-fed protein','plant based protein'
,'animal protein','protein','sugar','blood glucose','fiber','starch','refined sugar','refined carbs','simple carbs','complex carbs','carbs','white hdl fat','brown ldl fat','fats','prebiotic','enzymes','probiotic','microbiome','gut health']

def tokenize_text(words):
    tokenizer = RegexpTokenizer('[A-Za-z]\w+')
    all_tokens = tokenizer.tokenize(words)
    case_insensitive = [token.lower() for token in all_tokens]
    bigrams = list(ngrams(case_insensitive,2))
    joined = [' '.join(gram) for gram in bigrams]
    trigrams = list(ngrams(case_insensitive,3))
    tri_joined = [' '.join(gram) for gram in trigrams]
    case_insensitive.extend(joined)
    case_insensitive.extend(tri_joined)
    return set(case_insensitive)

def clean_tweets(data, categories):
    topics = []
    topics2 = []
    topics3 = []
    topics4 = []
    topics5 = []
    for tweet in data:
        categories_dict = {1:'null',2:'null',3:'null',4:'null',5:'null'}
        words = tokenize_text(tweet.text)
        print(words)
        intersection = list(words.intersection(categories))
        if len(intersection) > 0:
            for i in list(range(0,len(intersection))):
                categories_dict[i+1] = intersection[i]
        else:
            categories_dict[1] = 'general'
        topics.append(categories_dict[1])
        topics2.append(categories_dict[2])
        topics3.append(categories_dict[3])
        topics4.append(categories_dict[4])
        topics5.append(categories_dict[5])
    tweets = [{'title':tweet.id, 'date':tweet.created_at.date().strftime('%Y-%m-%d'),
       'description': tweet.text, 'source':tweet.user.screen_name,'source_id':tweet.user.id_str,
       'formality': 'Informal','difficulty':'Easy',
       'length': 1,'medium':'text'} for tweet in data]
    tweets = pd.DataFrame(tweets)
    tweets['param_1'] = topics
    tweets['param_2'] = topics2
    tweets['param_3'] = topics3
    tweets['param_4'] = topics4
    tweets['param_5'] = topics5
    return tweets




#CALL API
def twitter_api_call(list_handles, categories):
    empty = pd.DataFrame()
    for handle in list_handles:
        user_tweets = pd.DataFrame(clean_tweets(api.user_timeline(handle), categories))
        empty = empty.append(user_tweets, sort=True)
        print(handle)
    empty.title = empty.title.astype('str')
    empty['web_url'] = 'https://twitter.com/'+empty.source+'/status/'+empty.title
    empty['image_url'] = empty['web_url']
    return empty
import requests

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text


In [107]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from nltk.tokenize import RegexpTokenizer

In [123]:
def tokenize_text(words):
    tokenizer = RegexpTokenizer('[A-Za-z]\w+')
    all_tokens = tokenizer.tokenize(words)
    case_insensitive = [token.lower() for token in all_tokens]
    bigrams = list(ngrams(token,2))
    joined = [' '.join(gram) for gram in bigrams]
    case_insensitive.extend(joined)
    return set(case_insensitive)

# Youtube 

In [8]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import pprint
import pafy
import warnings
warnings.filterwarnings("ignore")
from apikeys import *
from info import *


DEVELOPER_KEY = youtube_key
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


def clean_youtube_time(string):
    if 'H' in string:
        minutes = int(string[string.find('H')+1:string.find('M')])
        hours = int(string[string.find('T')+1:string.find('H')]) * 60
        time = minutes + hours
    else:
        if 'M' in string:
            time = int(string[string.find('T')+1:string.find('M')])
        else:
            time = 1
    return time

def youtube_search(q, max_results=10,order="date", token=None, location=None, location_radius=None):

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

    search_response = youtube.search().list(
    q=q,
    type="video",
    pageToken=token,
    order = order,
    part="id,snippet", # Part signifies the different types of data you want
    maxResults=max_results,
    location=location,
    locationRadius=location_radius).execute()

    all_dicts = []

    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":

            title = (search_result['snippet']['title'])

            videoId = (search_result['id']['videoId'])

            response = youtube.videos().list(
            part='statistics, snippet, contentDetails',
            id=search_result['id']['videoId']).execute()

            channelId = (response['items'][0]['snippet']['channelId'])
            channelTitle = (response['items'][0]['snippet']['channelTitle'])
            categoryId = (response['items'][0]['snippet']['categoryId'])
            favoriteCount = (response['items'][0]['statistics']['favoriteCount'])
            viewCount = (response['items'][0]['statistics']['viewCount'])
            date = pd.to_datetime((response['items'][0]['snippet']['publishedAt'])).date().strftime('%Y-%m-%d')
            description = response['items'][0]['snippet']['localized']['description']
            url = 'https://www.youtube.com/watch?v='+videoId
            image_url = response['items'][0]['snippet']['thumbnails']['default']['url']
            length = clean_youtube_time(response['items'][0]['contentDetails']['duration'])

        if 'commentCount' in response['items'][0]['statistics'].keys():
            commentCount = (response['items'][0]['statistics']['commentCount'])
        else:
            commentCount = []

        if 'tags' in response['items'][0]['snippet'].keys():
            tags = (response['items'][0]['snippet']['tags'])
        else:
            tags = []

        youtube_dict = {'tags':tags,'source_id': channelId,'source': channelTitle,'categoryId':categoryId,'title':title,'videoId':videoId,'viewCount':viewCount,'commentCount':commentCount,'favoriteCount':favoriteCount,
                        'formality':'Intermediate', 'medium':'video','date':date, 'description': description, 'web_url':url, 'image_url':image_url, 'length':length}
        all_dicts.append(youtube_dict)
    return pd.DataFrame(all_dicts)

def add_category(df, categories):
    # cats = ['keto','ketogenic','paleo','paleolithic','vegan','vegetarian']
    all_params = []
    for index, row in df.iterrows():
        try:
            intersect = list(set(row.tags).intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = row.tags[0]
        except:
            category = 'none'
        all_params.append(category)
    df['param'] = all_params
    return df

def youtube_api_call(list_accounts, categories):
    empty_df = pd.DataFrame()
    errors = []
    for account in list_accounts:
        try:
            df = youtube_search(account)
            df = add_category(df, categories)
            empty_df = empty_df.append(df, sort=True)
            print(account)
        except:
            print(account + " EXCEPTION!!!!")
    return empty_df

# FB AND INSTA

In [156]:
wall_search = "https://graph.facebook.com/v3.2/WellNYT/feed/?access_token=EAACl5okwUhQBAAKh8AmW6y0njZBuMKZAZB8ANIrdMGKQoySrLXKBfZANfL6n1h0ePv4GdzzZA1cLfOfUbeYervGzZCjB0NqpGXZBXScxCz7ZA9jGoxmEVWVAcGX1hq92EDV3zHeGIxOoWIEZAQmbkQsQdbKNaIXI0rqAubLHvrHwNGwIPRI29rz9gLYSrVDQ7RsMZD"

In [157]:
response = requests.get(wall_search)


In [158]:
response.json()

{'error': {'message': "(#10) To use 'Page Public Content Access', your use of this endpoint must be reviewed and approved by Facebook. To submit this 'Page Public Content Access' feature for review please read our documentation on reviewable features: https://developers.facebook.com/docs/apps/review.",
  'type': 'OAuthException',
  'code': 10,
  'fbtrace_id': 'DRIOJeRua/e'}}

In [28]:


nyt_fb_post = ['https://www.facebook.com/WellNYT/posts/2030812243705795', 'https://www.facebook.com/WellNYT/posts/2027922970661389',
              'https://www.facebook.com/WellNYT/posts/2027920193995000','https://www.facebook.com/WellNYT/posts/2023650137755339',
              'https://www.facebook.com/WellNYT/posts/2022428551210831','https://www.facebook.com/WellNYT/posts/2022404617879891']

nyt_fb_vid = ['https://www.facebook.com/WellNYT/videos/vl.215269862225520/1148312565289105/?type=1','https://www.facebook.com/WellNYT/videos/vl.215269862225520/1146634572123571/?type=1',
         'https://www.facebook.com/WellNYT/videos/vl.215269862225520/1146398342147194/?type=1', 'https://www.facebook.com/WellNYT/videos/1437656589688033/'
         ,'https://www.facebook.com/WellNYT/videos/1408624012591291/','https://www.facebook.com/WellNYT/videos/1390021831118176/','https://www.facebook.com/WellNYT/videos/1363150167138676/',
         'https://www.facebook.com/WellNYT/videos/1362877763832583/']

def random_cat(list_categories, length):
    cats = []
    for i in list(range(0,length)):
        cats.append(random.choice(list_categories))
    return cats

def random_length(length):
    lengths = []
    for i in list(range(0,length)):
        lengths.append(random.randint(0, 15))
    return lengths


categories = ['vegan','keto','vegetarian','paleo']

def create_sample_fb(list_urls, categories_list, medium_type):
    df = pd.DataFrame()
    df['web_url'] = list_urls
    df['image_url'] = list_urls
    df['description'] = 'Facebook Dummy Data'
    df['title'] = 'Facebook Dummy Data'
    df['date'] = '2018-12-26'
    df['medium'] = medium_type
    df['source'] = 'NYT Health'
    df['source_id'] = 'Facebook'
    df['formality'] = 'Informal'
    df['difficulty'] = 'Medium'
    length = len(df.web_url)
    df['length'] = random_length(length)
    df['param'] = random_cat(categories_list,length)
    return df
    

In [31]:
insta_post = ['https://www.instagram.com/p/Br2LWE1hHOc/','https://www.instagram.com/p/Br2LHsDB4rL/',
                 'https://www.instagram.com/p/Br2K-V8heFw/','https://www.instagram.com/p/Br1R6cjBI5j/',
                 'https://www.instagram.com/p/Br1RjfphbWn/','https://www.instagram.com/p/Br1RYYthKBq/',
                 'https://www.instagram.com/p/Bruqo-ihpaH/','https://www.instagram.com/p/BruqhH9BlJ0/',
                 'https://www.instagram.com/p/Brrp-7PBqlf/','https://www.instagram.com/p/BrrpzXIBbgC/','https://www.instagram.com/p/Brmjo72ByfA/',
                 'https://www.instagram.com/p/Brmjeg4huS_/','https://www.instagram.com/p/BrmjRICBXVk/','https://www.instagram.com/p/BrmhYKiBli5/']

def create_sample_insta(list_urls, categories_list):
    df = pd.DataFrame()
    df['web_url'] = list_urls
    df['image_url'] = list_urls
    df['description'] = 'Instagram Dummy Data'
    df['title'] = 'Instagram Dummy Data'
    df['date'] = '2018-12-26'
    df['source'] = 'Daily Health Tips'
    df['source_id'] = 'Instagram'
    df['formality'] = 'Informal'
    df['difficulty'] = 'Easy'
    df['medium'] = 'video'
    length = len(df.web_url)
    df['length'] = random_length(length)
    df['param'] = random_cat(categories_list,length)
    return df

# AUDIO

In [299]:
import requests
import json
import feedparser
import pandas as pd
import re
import time


def feed_urls(search_words, media_value='podcast', entity_value='podcast'):
#     Args:
#         search_words: The URL-encoded text string to be searched for
#         media_value: {movie, podcast, music, musicVideo, audiobook,
#                     shortFilm, tvShow, software, ebook, all} optional
#                     An optional variable, which indicates the media type to be searched for.
#         entity_value: Optional

    payload = {'term': search_words, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        feed_url = itunes_result_json["results"][0]['feedUrl']
    else:
        feed_url = "None"
    return feed_url

def fix_podcast_length(time):
    hours = int(time[0:2]) * 60
    minutes = int(time[4:5])
    length = hours + minutes
    return length

def df_podcast_episodes(feed_url):
    print(feed_url)
    if(len(feed_url) > 0):
        feed = feedparser.parse(feed_url)
        episodes = []
        for episode in feed.entries:
            info = dict()
            info['title'] = episode['title'] if 'title' in episode else ''
            info['description']= episode['summary'] if 'summary' in episode else ''
            info['length']= fix_podcast_length(episode['itunes_duration']) if 'itunes_duration' in episode else 1
            info['date']= pd.to_datetime(episode['published']).date().strftime('%Y-%m-%d') if 'published' in episode else ''
            info['medium'] = 'audio'
            info['formality'] = 'Intermediate'
            info['source'] = feed['feed']['title'] if 'title' in feed['feed'] else ''
            info['source_id'] = feed['feed']['title_detail']['base'] if 'title_detail' in feed['feed'] else ''
            try:
                info['web_url'] = episode['links'][0]['href'] if 'links' in episode else ''
            except:
                info['web_url'] = feed['feed']['title_detail']['base'] if 'title_detail' in feed['feed'] else ''
            info['image_url'] = feed['feed']['image']['href'] if 'image' in feed['feed'] else ''
            episodes.append(info)
        df = pd.DataFrame(episodes)
        return df
    else:
        print("No response!")

def add_category_to_audio(df, categories):
    all_params = []
    for index, row in df.iterrows():
        try:
            words = set(re.sub("[^\w]", " ",  row.description).split())
            intersect = list(words.intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = 'general'
        except:
            category = 'general'
        all_params.append(category)
    df['param'] = all_params
    return df

def call_podcast_api(categories, podcasts):
    empty = pd.DataFrame()
    for podcast in podcasts:
        if podcast == 'None':
            pass
        else:
            try:
                url = feed_urls(podcast)
                df = df_podcast_episodes(url)
                df = add_category_to_audio(df, categories)
                empty = empty.append(df, sort=True)
                print('SUCCESS!')
                time.sleep(2)
            except:
                print('EXCEPTION')
                time.sleep(2)
    return empty


# EBOOKS

In [334]:
pd.set_option('display.max_columns', None)

In [368]:
def clean_ebook_date(df):
    dates = []
    for index, row in df.iterrows():
        date = pd.to_datetime(row.date).date().strftime('%Y-%m-%d')
        dates.append(date)
    df['date'] = dates
    return df


def ebook_search(search_word, media_value='ebook', entity_value='ebook'):
    payload = {'term': search_word, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        df = pd.DataFrame(itunes_result_json['results'])
        #NOTE LENGTH IS ACTUALLY THE PRICE BUT USE SAME LABEL FOR CONSISTENCY
        df = df.rename(index = str, columns= {'artistName': 'source','trackViewUrl':'web_url',
                                        'artworkUrl100':'image_url','price': 'length','releaseDate':'date','trackName':'title'})
        df['source_id'] = 'Itunes Ebook'
        df['formality'] = 'Formal'
        df['medium'] = 'text'
        df['param'] = search_word
        df = df.fillna(1)
        return clean_ebook_date(df)
    else:
        print('No Results!')
        return 'Empty'

def call_ebook_api(categories):
    empty = pd.DataFrame()
    for category in categories:
        df = ebook_search(category)
        if type(df) != str:
            empty = empty.append(df, sort=True)
            time.sleep(2)
            print('Added '+category)
        else:
            pass
    return empty


# Movies/Docs

In [472]:
def add_category_to_movie(df, categories):
    all_params = []
    for index, row in df.iterrows():
        try:
            words = set(re.sub("[^\w]", " ",  row.description).split())
            intersect = list(words.intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = 'general'
        except:
            category = 'general'
        all_params.append(category)
    df['param'] = all_params
    return df

def movie_search(search_word,categories, media_value='movie', entity_value='movie'):
    payload = {'term': search_word, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        df = pd.DataFrame(itunes_result_json['results'])
        df = df.rename(index = str, columns= {'artistName': 'source','trackViewUrl':'web_url',
                                        'artworkUrl100':'image_url','trackTimeMillis': 'length'
                                              ,'releaseDate':'date','trackName':'title', 'longDescription':'description'})
        df['source_id'] = 'Itunes Movie'
        df['formality'] = 'Formal'
        df['medium'] = 'video'
        df['length'] = round(df['length'] / 60000)
        df = df.fillna(1)
        df = clean_ebook_date(df)
        return add_category_to_movie(df,categories)
    else:
        print('No Results!')
        return 'Empty'

def call_movie_api(movies, categories):
    empty = pd.DataFrame()
    for movie in movies:
        try:
            df = movie_search(movie, categories)
            if type(df) != str:
                empty = empty.append(df, sort=True)
                time.sleep(5)
                print('Added '+movie)
            else:
                time.sleep(5)
        except:
            print(movie + ' EXCEPTION')
    return empty


# AUDIOBOOKS

In [487]:

def audiobook_search(search_word, media_value='audiobook', entity_value='audiobook'):
    payload = {'term': search_word, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        df = pd.DataFrame(itunes_result_json['results'])
        #NOTE LENGTH IS ACTUALLY THE PRICE BUT USE SAME LABEL FOR CONSISTENCY
        df = df.rename(index = str, columns= {'artistName': 'source','collectionViewUrl':'web_url',
                                        'artworkUrl100':'image_url','collectionPrice': 'length','releaseDate':'date','collectionName':'title'})
        df['source_id'] = 'Itunes Audiobook'
        df['formality'] = 'Formal'
        df['medium'] = 'audio'
        df['param'] = search_word
        df = df.fillna(1)
        return clean_ebook_date(df)
    else:
        print('No Results!')
        return 'Empty'

def call_audiobook_api(categories):
    empty = pd.DataFrame()
    for category in categories:
        df = audiobook_search(category)
        if type(df) != str:
            empty = empty.append(df, sort=True)
            time.sleep(2)
            print('Added '+category)
        else:
            pass
    return empty

# UDEMY & COURSERA

In [166]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup

In [ ]:
def pull_pods(parameter):
    list_dicts = []
    urls = []
    split = parameter.split()
    if len(split) < 2:
        split.append('')
    html_page = requests.get('https://tunein.com/search/?query='+split[0]+'%'+'20'+split[1])
    soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding='utf-8')
    pods = soup.findAll('a')
    strings = []
    for number in list(range(0,10)):
        strings.append(str(number))
    for pod in pods[6:]:
        if pod.get('href') is not None:
            if pod.get('href')[-1] in strings:
                url = "https://tunein.com/embed/player/t"+pod.get('href')[-9:]+"/"
                data = {'author':'audio', 'content':'audio','description':'audio','publishedAt':'audio', 'source_id':'audio', 'source_name':'audio','title':'audio','url':url,'urlToImage':'audio', 'medium':'audio'}
                if url in urls:
                    pass
                else:
                    list_dicts.append(data)
                urls.append(url)
    return list_dicts

In [203]:
html_page = requests.get('https://www.everydayhealth.com/search/keto/?iid=gnav_head_search')
soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding='utf-8')
results = soup.findAll('a')
results = [result.get('href') for result in results]

In [206]:
results[50]

'//www.everydayhealth.com/ketogenic-diet/diet/types-targeted-keto-high-protein-keto-keto-cycling-more/'

In [209]:
web = requests.get('https:'+results[50])
soup = BeautifulSoup(web.content, 'html.parser', from_encoding='utf-8')
# results = soup.findAll('a')

In [215]:
print(soup.title.string)

Ketogenic Diet Types: Targeted Keto, High-Protein Keto, Keto Cycling, More | Everyday Health


In [216]:
soup.findAll('p')

[<p class="clear"></p>,
 <p>Sign up for more <a href="//www.everydayhealth.com/newsletter-subscriptions/signup">FREE Everyday Health newsletters</a>.</p>,
 <p class="newsletter-privacy">We respect <a href="/privacyterms/#everyday_health_privacy_policy" target="_blank">your privacy</a>.</p>,
 <p>Some ketogenic (or “keto”) diet devotees stay true to the diet 100 percent of the time, while others have found they need a little more carbohydrates or protein. That’s inspired some to tweak the low-carb, high-fat <a href="//www.everydayhealth.com/diet-nutrition/best-worst-diet-plans-for-weight-loss/">diet</a> to meet their needs. As a result, several spins on the <a href="//www.everydayhealth.com/diet-nutrition/ketogenic-diet/">keto diet</a> have emerged.<p>Kristen Kizer, RD, a registered clinical dietitian at Houston Methodist Hospital in Houston, says that all of these diets have one thing in common. “A keto diet to me would be any diet that gets a body into <a href="//www.everydayhealth.com

In [191]:
results[20].get('href')

'/high-cholesterol/?iid=gnav_nav_high'

In [173]:
requests.get('https://www.coursera.org/specializations/python')

<Response [200]>